In [1]:
import os
import pandas as pd
import xlrd

In [95]:
### Clean web of science articles

wos_ini = pd.read_excel('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/WOS_export2020-09-26.xls')

# Get only J type publications and only these fields: 'Author Full Names', 'Article Title',
# 'Source Title', 'Abstract', 'Publication Date', 'Publication Year'
df_req = df_all.query('`Publication Type` == "J"').loc[:, ['Article Title', 'Publication Year',
                                                           'Source Title', 'Abstract', 
                                                           'Author Keywords']]

# artTitle, pubYear , pubJournal, artAbstract
df_req.rename(columns={'Article Title': 'artTitle', 'Publication Year': 'pubYear', 
                       'Source Title': 'pubJournal', 'Abstract': 'artAbstract',
                      'Author Keywords': 'artKeywords'}, inplace=True)
print(list(df_req))
print(df_req.shape[0])
df_req.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/db_req_fields_articles/bda_ml_wm_wos_req_s1.csv', index=False)

['artTitle', 'pubYear', 'pubJournal', 'artAbstract', 'artKeywords']
325


In [96]:
#### Clean proquest articles
pro_df = pd.read_excel('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/ProQuestDocuments-2020-09-26.xls')

pro_df_req = pro_df.query('documentType != "Feature"').loc[:, ['Title', 'pubdate', 'pubtitle', 'Abstract', 'subjectTerms', 'subjects']]

pro_df_req['artKeywords'] = pro_df_req['subjectTerms'] + pro_df_req['subjects']

# artTitle, pubYear , pubJournal, artAbstract
pro_df_req.rename(columns={'Title': 'artTitle', 'pubdate': 'pubYear', 'pubtitle': 'pubJournal', 'Abstract': 'artAbstract'}, inplace=True)

pro_df_req = pro_df_req.drop(['subjectTerms', 'subjects'], axis=1)

print(list(pro_df_req))
print(pro_df_req.shape[0])
pro_df_req.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/db_req_fields_articles/bda_ml_wm_pq_req_s1.csv', index=False)

['artTitle', 'pubYear', 'pubJournal', 'artAbstract', 'artKeywords']
143


In [106]:
### Clean science direct files
os.chdir('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/sci_dir_ini')

df = []

for fl in os.listdir():
    ## Without sep ='\t' option, it was throwing ParserError: Error tokenizing data. 
    #C error: Expected 31 fields in line 44, saw 32 (sci)_dir_6.csv
    # data = pd.read_csv('file1.csv', error_bad_lines=False) - works but just ignores the bad lines
    ## tried using sep='\t', loads correctly but then each row of df was just 1 continuous string
    # so manually removed offending line for now

    if fl != '.DS_Store':
        df.append(pd.read_csv(fl))

sci_dir_df = pd.concat(df, axis=0)

# Title, Journal, Year, Custom1
sci_dir_req_df = sci_dir_df.loc[:, ['Title', 'Year', 'Journal', 'Custom1', 'Custom3']]

# artTitle, pubYear, pubJournal, artAbstract
sci_dir_req_df.rename(columns={'Title': 'artTitle', 'Year': 'pubYear', 'Journal': 'pubJournal', 
                               'Custom1': 'artAbstract', 'Custom3': 'artKeywords'}, inplace=True)
print(list(sci_dir_req_df))
print(sci_dir_req_df.shape[0])
sci_dir_req_df.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/db_req_fields_articles/bda_ml_wm_sd_req_s1.csv', index=False)

['artTitle', 'pubYear', 'pubJournal', 'artAbstract', 'artKeywords']
614


In [107]:
### Clean ieee xplore

os.chdir('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/ieee_ini')

df = []

for fl in os.listdir():
    print(fl)
    if fl != '.DS_Store':
        df.append(pd.read_csv(fl))

ieee_df = pd.concat(df, axis=0)

ieee_df_req = ieee_df.loc[:, ['Document Title', 'Date Added To Xplore', 'Publication Title', 'Abstract', 'Author Keywords']]
ieee_df_req.rename(columns={'Document Title': 'artTitle', 'Date Added To Xplore': 'pubYear', 
                            'Publication Title': 'pubJournal', 'Abstract': 'artAbstract',
                            'Author Keywords': 'artKeywords'}, inplace=True)

print(list(ieee_df_req))
print(ieee_df_req.shape[0])

ieee_df_req.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/db_req_fields_articles/bda_ml_wm_ieee_req_s1.csv', index=False)

IEEE_ab_export2020-09-26.csv
IEEE_ti_export2020-09-26.csv
IEEE_kw_export2020-09-26.csv
['artTitle', 'pubYear', 'pubJournal', 'artAbstract', 'artKeywords']
115


In [8]:
### Append all db files to one
os.chdir('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/db_req_fields_articles')
df = []

for fl in os.listdir():
    df.append(pd.read_csv(fl))

all_dbs = pd.concat(df, axis=0)
print(all_dbs.shape[0])
all_dbs.drop_duplicates(subset='artTitle', keep='first', inplace=True)
print(all_dbs.shape[0])
         
### get ti+ab+kw combination column
# replace nans with empty strings or .
all_dbs.fillna('none', inplace=True)
all_dbs['ti_ab_kws'] = all_dbs['artTitle'] + all_dbs['artAbstract'] + all_dbs['artKeywords'] 
all_dbs.drop('Unnamed: 0', axis=1, inplace=True)
all_dbs.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/bda_ml_wm_all_db_ini_articles.csv', index=False)

1419
1204


### filter out based on g1+g2 combination matching with ti+ab+kw combination

In [3]:
### get g1+g2 combinations list
import itertools as it

group1 = ["waste" , "solid waste" , "liquid waste" , "waste water" , "e-waste" , 
          "recyclable waste" , "organic waste" , "food waste"]
group2 = ["big data" , "data mining" , "machine learning" , "artificial intelligence" , 
          "descriptive" , "predictive" , "prescriptive", "deep learning" , "neural network",
          "neural networks", "data visualization"]

comb_group = list(it.product(group1, group2))
print(comb_group)

[('waste', 'big data'), ('waste', 'data mining'), ('waste', 'machine learning'), ('waste', 'artificial intelligence'), ('waste', 'descriptive'), ('waste', 'predictive'), ('waste', 'prescriptive'), ('waste', 'deep learning'), ('waste', 'neural network'), ('waste', 'neural networks'), ('waste', 'data visualization'), ('solid waste', 'big data'), ('solid waste', 'data mining'), ('solid waste', 'machine learning'), ('solid waste', 'artificial intelligence'), ('solid waste', 'descriptive'), ('solid waste', 'predictive'), ('solid waste', 'prescriptive'), ('solid waste', 'deep learning'), ('solid waste', 'neural network'), ('solid waste', 'neural networks'), ('solid waste', 'data visualization'), ('liquid waste', 'big data'), ('liquid waste', 'data mining'), ('liquid waste', 'machine learning'), ('liquid waste', 'artificial intelligence'), ('liquid waste', 'descriptive'), ('liquid waste', 'predictive'), ('liquid waste', 'prescriptive'), ('liquid waste', 'deep learning'), ('liquid waste', 'neu

In [7]:
import pandas as pd

def check_ti_ab_kw_comb(val):
    count = 0
    for item in comb_group:
        if ((item[0].lower() in val.lower()) and (item[1].lower() in val.lower())):
            return True
    return False

all_dbs_df = pd.read_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/bda_ml_wm_all_db_ini_articles.csv')
print(all_dbs_df.shape[0])
all_dbs_df['req_paper'] = all_dbs_df['ti_ab_kws'].map(check_ti_ab_kw_comb)
print(all_dbs_df.shape[0])
all_dbs_df.to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/bda_ml_wm_all_db_tiabkw_true_false_articles.csv', index=False)
all_dbs_df.query('req_paper == True').to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/bda_ml_wm_all_db_tiabkw_true_articles.csv', index=False)
print(all_dbs_df.query('req_paper == True').shape[0])
all_dbs_df.query('req_paper == False').to_csv('/Users/salma/Research/papers_data_analysis/bda_ml_wm_lr/bda_ml_wm_all_db_tiabkw_false_articles.csv', index=False)
print(all_dbs_df.query('req_paper == False').shape[0])

1204
1204
651
553
